In [1]:
# Install the tsfm library
! pip install "granite-tsfm[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.3.1"

  Cloning https://github.com/ibm-granite/granite-tsfm.git (to revision v0.3.1) to /tmp/pip-install-w1f9zphk/granite-tsfm_2da4a6b606fb46c0a0b6d57f97a2619e
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite/granite-tsfm.git /tmp/pip-install-w1f9zphk/granite-tsfm_2da4a6b606fb46c0a0b6d57f97a2619e
  Running command git checkout -q 16106d70d1fb3244eecd48c8fbbf3a0009fb8751
  Resolved https://github.com/ibm-granite/granite-tsfm.git to commit 16106d70d1fb3244eecd48c8fbbf3a0009fb8751
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import math
import os
import tempfile

import pandas as pd
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public.toolkit import TimeSeriesPreprocessor, TrackingCallback, count_parameters, get_datasets
from tsfm_public.toolkit.get_model import get_model
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.visualization import plot_predictions

In [2]:
import warnings


# Suppress all warnings
warnings.filterwarnings("ignore")

### Important arguments

In [4]:
# Set seed for reproducibility
SEED = 42
set_seed(SEED)

# TTM Model path. The default model path is Granite-R2. Below, you can choose other TTM releases.
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"

# Context length, Or Length of the history.
# Currently supported values are: 512/1024/1536 for Granite-TTM-R2 and Research-Use-TTM-R2, and 512/1024 for Granite-TTM-R1
CONTEXT_LENGTH = 1536

# Granite-TTM-R2 supports forecast length upto 720 and Granite-TTM-R1 supports forecast length upto 96
PREDICTION_LENGTH = 168+216 # 9일치 + 168 추론

# Results dir
OUT_DIR = "dacon_mypsyche98_finetuned_models/"

# Data processing

In [5]:
# Dataset
TARGET_DATASET = "train"
dataset_path = "./train_test.csv"
timestamp_column = "일시"
id_columns = ['건물번호']  # mention the ids that uniquely identify a time-series.

control_columns = ["기온(°C)","강수량(mm)","풍속(m/s)","습도(%)"]
target_columns = ["전력소비량(kWh)"]
split_config = {
    "train": [288-216, 1824-216],        # 인덱스 288부터 1823까지 (총 1536개)
    "valid": [1824-216, 2040-216],       # 인덱스 1824부터 2039까지 (총 216개 = 9일)
    "test": [2040-216, 2040+168],  # 인덱스 2040부터 2207까지 (총 168개)
}
# Understanding the split config -- slides

data = pd.read_csv(
    dataset_path,
    parse_dates=[timestamp_column],
)

# --- 디버깅: ID 1번 데이터의 크기 확인 ---
building1_data = data[data['건물번호'] == 1]
print(f"건물번호 1번의 데이터 개수: {len(building1_data)}개")
# 이 값이 2208이 맞는지 확인해 주세요.

column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": target_columns,
    "control_columns": control_columns,
}

건물번호 1번의 데이터 개수: 2208개


## Zero-shot evaluation method

In [6]:
def zeroshot_eval(dataset_name, batch_size, context_length=512, forecast_length=96):
    # Get data
    tsp = TimeSeriesPreprocessor(
        **column_specifiers,
        context_length=context_length,
        prediction_length=forecast_length,
        scaling=True,
        encode_categorical=False,
        scaler_type="standard",
    )
    # Load model
    zeroshot_model = get_model(
        TTM_MODEL_PATH,
        context_length=context_length,
        prediction_length=forecast_length,
        freq_prefix_tuning=False,
        freq=None,
        prefer_l1_loss=False,
        prefer_longer_context=True,
    )
    dset_train, dset_valid, dset_test = get_datasets(
        tsp, data, split_config, use_frequency_token=zeroshot_model.config.resolution_prefix_tuning
    )
    print("dset_train=",dset_train.group_df.head())
    print("dset_valid=",dset_valid.head())
    print("dset_test=",dset_test.head())
    temp_dir = tempfile.mkdtemp()
    # zeroshot_trainer
    zeroshot_trainer = Trainer(
        model=zeroshot_model,
        args=TrainingArguments(
            output_dir=temp_dir,
            per_device_eval_batch_size=batch_size,
            seed=SEED,
            report_to="none",
        ),
    )
    # evaluate = zero-shot performance
    print("+" * 20, "Test MSE zero-shot", "+" * 20)
    zeroshot_output = zeroshot_trainer.evaluate(dset_test)
    print("zeroshot_output=", zeroshot_output)
    # get predictions
    predictions_dict = zeroshot_trainer.predict(dset_test)
    predictions_np = predictions_dict.predictions[0]
    print("predictions_np.shape=",predictions_np.shape)
    # get backbone embeddings (if needed for further analysis)
    backbone_embedding = predictions_dict.predictions[1]
    print("backbone_embedding.shape=",backbone_embedding.shape)
    # plot
    """
    plot_predictions(
        model=zeroshot_trainer.model,
        dset=dset_test,
        plot_dir=os.path.join(OUT_DIR, dataset_name),
        plot_prefix="test_zeroshot",
        indices=[685, 118, 902, 1984, 894, 967, 304, 57, 265, 1015],
        channel=0,
    )
    """
    # zeroshot_trainer와 tsp를 함께 반환
    return dset_train, dset_valid, dset_test, predictions_dict, zeroshot_trainer, tsp

# Zeroshot

In [7]:
# 1. 수정된 zeroshot_eval 함수 호출
dset_train, dset_valid, dset_test, predictions_dict, zeroshot_trainer, tsp = zeroshot_eval(
    dataset_name=TARGET_DATASET, context_length=CONTEXT_LENGTH, forecast_length=PREDICTION_LENGTH, batch_size=64
)

INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Loading model from: ibm-granite/granite-timeseries-ttm-r2


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/20.0M [00:00<?, ?B/s]

INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 1536-720-r2.
INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:[TTM] context_length = 1536, prediction_length = 720


dset_train= <tsfm_public.toolkit.dataset.ForecastDFDataset object at 0x7b64d9b25100>
dset_valid= <tsfm_public.toolkit.dataset.ForecastDFDataset object at 0x7b64d96123c0>
dset_test= <tsfm_public.toolkit.dataset.ForecastDFDataset object at 0x7b64d9797710>
++++++++++++++++++++ Test MSE zero-shot ++++++++++++++++++++


zeroshot_output= {'eval_loss': 0.4275771677494049, 'eval_model_preparation_time': 0.0043, 'eval_runtime': 3.6973, 'eval_samples_per_second': 27.047, 'eval_steps_per_second': 0.541}
predictions_np.shape= (100, 384, 5)
backbone_embedding.shape= (100, 5, 12, 384)


In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_predictions_from_eval(
    data, dset_test, predictions_dict, split_config, building_id, forecast_length, tsp
):
    """
    여러 빌딩의 데이터를 사용하여 예측 결과를 시각화합니다.
    Args:
        data (pd.DataFrame): 모든 건물 데이터가 합쳐진 원본 DataFrame.
        dset_test (Dataset): zeroshot_eval에서 반환된 테스트 데이터셋.
        predictions_dict (dict): zeroshot_eval에서 반환된 예측 결과.
        split_config (dict): 데이터 분할 설정.
        building_id (int): 시각화할 건물 번호.
        forecast_length (int): 예측 길이.
        tsp (TimeSeriesPreprocessor): 데이터 전처리 객체.
    """
    # 데이터셋의 총 row 개수를 기준으로 각 빌딩의 시작점과 끝점을 계산합니다.
    # 예: 건물번호 1-100, 총 데이터 204000개
    rows_per_building = data.shape[0] // 100 # 총 100개 건물로 가정
    start_index_in_combined = (building_id - 1) * rows_per_building
    end_index_in_combined = building_id * rows_per_building

    # 1. 모델에 입력된 과거 데이터(context) 추출
    start_test_idx_in_building = split_config['test'][0]
    context_length = split_config['valid'][0] - split_config['train'][0]

    historical_df = data.iloc[start_index_in_combined + start_test_idx_in_building - context_length :
                              start_index_in_combined + start_test_idx_in_building].copy()

    # 2. 예측 기간에 해당하는 실제값(정답) 데이터 추출
    end_test_idx_in_building = split_config['test'][1]
    ground_truth_df = data.iloc[start_index_in_combined + start_test_idx_in_building :
                                start_index_in_combined + end_test_idx_in_building].copy()

    # 3. predictions_dict에서 예측 결과 추출
    # predictions_dict의 첫 번째 차원은 빌딩 ID에 해당합니다.
    predictions_for_plot = predictions_dict.predictions[0][building_id - 1, :, 0]

    # --- 언스케일링 과정 ---
    # `target_scaler_dict`의 키는 빌딩 ID입니다.
    target_scaler = tsp.target_scaler_dict[building_id]
    predictions_for_plot = predictions_for_plot.reshape(-1, 1)
    unscaled_predictions = target_scaler.inverse_transform(predictions_for_plot).flatten()

    # 4. 데이터 시각화
    plt.figure(figsize=(15, 6))

    # 과거 데이터와 미래의 실제값을 합쳐서 '실제값' 라인으로 플롯
    full_ground_truth_df = pd.concat([historical_df, ground_truth_df])
    plt.plot(full_ground_truth_df['일시'], full_ground_truth_df['전력소비량(kWh)'], label='실제값', color='blue')

    # 예측값을 플롯
    forecast_timestamps = pd.date_range(start=historical_df['일시'].iloc[-1], periods=forecast_length + 1, freq='H')[1:]
    plt.plot(forecast_timestamps, unscaled_predictions, label='예측값', color='red', linestyle='--')

    # 제목 및 라벨
    plt.title(f'건물 {building_id}의 전력소비량 예측', fontsize=16)
    plt.xlabel('일시', fontsize=12)
    plt.ylabel('전력소비량 (kWh)', fontsize=12)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [9]:
for i in range(1,101,1):
    plot_predictions_from_eval(
        data=data,
        dset_test=dset_test,
        predictions_dict=predictions_dict,
        split_config=split_config,
        building_id=i,
        forecast_length=PREDICTION_LENGTH,
        tsp=tsp
    )

Output hidden; open in https://colab.research.google.com to view.

Few-shot finetune and evaluation method

In [12]:
def fewshot_finetune_eval(
    dataset_name,
    batch_size,
    learning_rate=None,
    context_length=512,
    forecast_length=96,
    fewshot_percent=5,
    freeze_backbone=True,
    num_epochs=50,
    save_dir=OUT_DIR,
    loss="mse",
    quantile=0.5,
):
    out_dir = os.path.join(save_dir, dataset_name)

    print("-" * 20, f"Running few-shot {fewshot_percent}%", "-" * 20)

    # Data prep: Get dataset

    tsp = TimeSeriesPreprocessor(
        **column_specifiers,
        context_length=context_length,
        prediction_length=forecast_length,
        scaling=True,
        encode_categorical=False,
        scaler_type="standard",
    )

    # change head dropout to 0.7 for ett datasets
    if "ett" in dataset_name:
        finetune_forecast_model = get_model(
            TTM_MODEL_PATH,
            context_length=context_length,
            prediction_length=forecast_length,
            freq_prefix_tuning=False,
            freq=None,
            prefer_l1_loss=False,
            prefer_longer_context=True,
            # Can also provide TTM Config args
            head_dropout=0.7,
            loss=loss,
            quantile=quantile,
        )
    else:
        finetune_forecast_model = get_model(
            TTM_MODEL_PATH,
            context_length=context_length,
            prediction_length=forecast_length,
            freq_prefix_tuning=False,
            freq=None,
            prefer_l1_loss=False,
            prefer_longer_context=True,
            # Can also provide TTM Config args
            loss=loss,
            quantile=quantile,
        )

    dset_train, dset_val, dset_test = get_datasets(
        tsp,
        data,
        split_config,
        fewshot_fraction=fewshot_percent / 100,
        fewshot_location="first",
        use_frequency_token=finetune_forecast_model.config.resolution_prefix_tuning,
    )

    if freeze_backbone:
        print(
            "Number of params before freezing backbone",
            count_parameters(finetune_forecast_model),
        )

        # Freeze the backbone of the model
        for param in finetune_forecast_model.backbone.parameters():
            param.requires_grad = False

        # Count params
        print(
            "Number of params after freezing the backbone",
            count_parameters(finetune_forecast_model),
        )

    # Find optimal learning rate
    # Use with caution: Set it manually if the suggested learning rate is not suitable
    if learning_rate is None:
        learning_rate, finetune_forecast_model = optimal_lr_finder(
            finetune_forecast_model,
            dset_train,
            batch_size=batch_size,
        )
        print("OPTIMAL SUGGESTED LEARNING RATE =", learning_rate)

    print(f"Using learning rate = {learning_rate}")
    finetune_forecast_args = TrainingArguments(
        output_dir=os.path.join(out_dir, "output"),
        overwrite_output_dir=True,
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        do_eval=True,
        eval_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        dataloader_num_workers=8,
        report_to="none",
        save_strategy="epoch",
        logging_strategy="epoch",
        save_total_limit=1,
        logging_dir=os.path.join(out_dir, "logs"),  # Make sure to specify a logging directory
        load_best_model_at_end=True,  # Load the best model when training ends
        metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
        greater_is_better=False,  # For loss
        seed=SEED,
    )

    # Create the early stopping callback
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=10,  # Number of epochs with no improvement after which to stop
        early_stopping_threshold=1e-5,  # Minimum improvement required to consider as improvement
    )
    tracking_callback = TrackingCallback()

    # Optimizer and scheduler
    optimizer = AdamW(finetune_forecast_model.parameters(), lr=learning_rate)
    scheduler = OneCycleLR(
        optimizer,
        learning_rate,
        epochs=num_epochs,
        steps_per_epoch=math.ceil(len(dset_train) / (batch_size)),
    )

    finetune_forecast_trainer = Trainer(
        model=finetune_forecast_model,
        args=finetune_forecast_args,
        train_dataset=dset_train,
        eval_dataset=dset_val,
        callbacks=[early_stopping_callback, tracking_callback],
        optimizers=(optimizer, scheduler),
    )
    finetune_forecast_trainer.remove_callback(INTEGRATION_TO_CALLBACK["codecarbon"])

    # Fine tune
    finetune_forecast_trainer.train()

    # Evaluation
    print("+" * 20, f"Test MSE after few-shot {fewshot_percent}% fine-tuning", "+" * 20)

    finetune_forecast_trainer.model.loss = "mse"  # fixing metric to mse for evaluation

    fewshot_output = finetune_forecast_trainer.evaluate(dset_test)
    print(fewshot_output)
    print("+" * 60)

    # get predictions

    predictions_dict = finetune_forecast_trainer.predict(dset_test)

    predictions_np = predictions_dict.predictions[0]

    print(predictions_np.shape)

    # get backbone embeddings (if needed for further analysis)

    backbone_embedding = predictions_dict.predictions[1]

    print(backbone_embedding.shape)

    # finetune_forecast_trainerr과 tsp를 함께 반환
    return dset_train, dset_valid, dset_test, predictions_dict, finetune_forecast_trainer, tsp

Few-shot 5%

In [13]:
dset_train, dset_valid, dset_test, predictions_dict, finetune_forecast_trainer, tsp = fewshot_finetune_eval(
    dataset_name=TARGET_DATASET,
    context_length=CONTEXT_LENGTH,
    forecast_length=PREDICTION_LENGTH,
    batch_size=64,
    fewshot_percent=5,
    learning_rate=0.001,
)

INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Loading model from: ibm-granite/granite-timeseries-ttm-r2


-------------------- Running few-shot 5% --------------------


INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 1536-720-r2.
INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:[TTM] context_length = 1536, prediction_length = 720


Number of params before freezing backbone 4998672
Number of params after freezing the backbone 2972112
Using learning rate = 0.001


Epoch,Training Loss,Validation Loss
1,0.840600,0.580433
2,0.711000,0.572158
3,0.616000,0.582150
4,0.563900,0.563018
5,0.522000,0.497459
6,0.474000,0.471732
7,0.423600,0.481171
8,0.384100,0.448305
9,0.364100,0.444419
10,0.329600,0.443852


[TrackingCallback] Mean Epoch Time = 5.315421998500824 seconds, Total Train Time = 151.8018991947174
++++++++++++++++++++ Test MSE after few-shot 5% fine-tuning ++++++++++++++++++++


{'eval_loss': 0.4698255658149719, 'eval_runtime': 2.7941, 'eval_samples_per_second': 35.79, 'eval_steps_per_second': 0.716, 'epoch': 20.0}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
(100, 384, 5)
(100, 5, 12, 384)


In [14]:
for i in range(1,101,1):
    plot_predictions_from_eval(
        data=data,
        dset_test=dset_test,
        predictions_dict=predictions_dict,
        split_config=split_config,
        building_id=i,
        forecast_length=PREDICTION_LENGTH,
        tsp=tsp
    )

Output hidden; open in https://colab.research.google.com to view.

Fewshot with quantile loss (We can use pinball loss to generate different quantiles as required)

In [22]:
dset_train, dset_valid, dset_test, predictions_dict, finetune_forecast_trainer, tsp = fewshot_finetune_eval(
    dataset_name=TARGET_DATASET,
    context_length=CONTEXT_LENGTH,
    forecast_length=PREDICTION_LENGTH,
    batch_size=64,
    fewshot_percent=5,
    learning_rate=0.001,
    loss="pinball",
    quantile=0.5,
)

INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Loading model from: ibm-granite/granite-timeseries-ttm-r2


-------------------- Running few-shot 5% --------------------


INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 1536-720-r2.
INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:[TTM] context_length = 1536, prediction_length = 720


Number of params before freezing backbone 4998672
Number of params after freezing the backbone 2972112
Using learning rate = 0.001


Epoch,Training Loss,Validation Loss
1,0.324900,0.270534
2,0.291700,0.264990
3,0.265100,0.263383
4,0.250800,0.256849
5,0.232500,0.233992
6,0.215800,0.222638
7,0.200400,0.230136
8,0.190000,0.220822
9,0.181700,0.214983
10,0.169000,0.213412


[TrackingCallback] Mean Epoch Time = 5.3700871706008915 seconds, Total Train Time = 156.65170216560364
++++++++++++++++++++ Test MSE after few-shot 5% fine-tuning ++++++++++++++++++++


{'eval_loss': 0.47490519285202026, 'eval_runtime': 2.6178, 'eval_samples_per_second': 38.2, 'eval_steps_per_second': 0.764, 'epoch': 20.0}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
(100, 384, 5)
(100, 5, 12, 384)


In [23]:
for i in range(1,101,1):
    plot_predictions_from_eval(
        data=data,
        dset_test=dset_test,
        predictions_dict=predictions_dict,
        split_config=split_config,
        building_id=i,
        forecast_length=PREDICTION_LENGTH,
        tsp=tsp
    )

Output hidden; open in https://colab.research.google.com to view.

In [24]:
import numpy as np

# 각 빌딩의 Descaling된 예측 결과를 담을 빈 리스트를 생성합니다.
all_descaled_predictions = []

# predictions_dict에서 각 빌딩의 예측 결과를 추출하고,
# tsp 객체에서 해당 빌딩의 스케일러를 사용하여 Descaling을 수행합니다.
for building_id in range(1, 101):
    # predictions_dict의 인덱스는 0부터 시작하므로 building_id - 1을 사용합니다.

    # 384개의 예측 결과 중 뒤쪽 168개만 선택합니다.
    # [시작인덱스:끝인덱스], -168은 뒤에서 168번째부터 끝까지를 의미합니다.
    predictions_for_building = predictions_dict.predictions[0][building_id - 1, -168:, 0]

    # Descaling을 위해 해당 빌딩의 스케일러를 가져옵니다.
    target_scaler = tsp.target_scaler_dict[building_id]

    # 예측 배열의 차원을 변환 (reshape)하여 스케일러에 전달할 수 있도록 준비합니다.
    predictions_for_plot = predictions_for_building.reshape(-1, 1)

    # Descaling을 수행합니다.
    descaled_predictions = target_scaler.inverse_transform(predictions_for_plot).flatten()

    # Descaling된 예측 결과를 리스트에 추가합니다.
    all_descaled_predictions.append(descaled_predictions)

# 리스트에 담긴 모든 Descaling된 예측 결과를 하나의 NumPy 배열로 합칩니다.
combined_descaled_predictions = np.concatenate(all_descaled_predictions)

# 결과 확인
# 각 빌딩별로 168개의 예측 결과가 있으므로, 총 길이는 100 * 168 = 16800개가 됩니다.
print(f"Descaling 후 합쳐진 예측 배열의 크기: {len(combined_descaled_predictions)}개")
print("Descaling 후 합쳐진 예측 배열의 첫 10개 값:", combined_descaled_predictions[:10])

Descaling 후 합쳐진 예측 배열의 크기: 16800개
Descaling 후 합쳐진 예측 배열의 첫 10개 값: [5348.567  4979.1836 4751.473  4586.5947 4549.5527 4801.6694 5016.787
 5742.5635 6354.1035 6794.012 ]


In [25]:
print(f"최대 예측값: {np.max(all_descaled_predictions[0])}")

print(f"최소 예측값: {np.min(all_descaled_predictions[0])}")

최대 예측값: 7836.21826171875
최소 예측값: 4462.5830078125


In [26]:
sample = pd.read_csv('./sample_submission.csv')

In [27]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num_date_time  16800 non-null  object
 1   answer         16800 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 262.6+ KB


In [28]:
# 위 DataFrame이 'submission_df'라는 변수에 저장되어 있다고 가정합니다.
# 'combined_descaled_predictions' 배열은 이미 준비되어 있습니다.
sample['answer'] = combined_descaled_predictions

# 변경사항을 확인합니다.
print(sample.head())
print(sample.tail())

   num_date_time       answer
0  1_20240825 00  5348.566895
1  1_20240825 01  4979.183594
2  1_20240825 02  4751.473145
3  1_20240825 03  4586.594727
4  1_20240825 04  4549.552734
         num_date_time       answer
16795  100_20240831 19  2546.321533
16796  100_20240831 20  2564.390137
16797  100_20240831 21  2603.178467
16798  100_20240831 22  2985.284668
16799  100_20240831 23  3214.335449


In [29]:
sample.to_csv('./baseline_submission_20250821_02.csv', index=False)